<a href="https://colab.research.google.com/github/Joseikwang/Time-Series/blob/main/Customer_Care_Chatbot_with_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Step1: Set up**

In [ ]:
# Install required packages
!pip install streamlit streamlit-chat numpy tensorflow pickle-mixin
!pip install pyngrok

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=5988 sha256=cea02b0ffd35365b1a1c247c3d29f05d45e44cc7e9a663d80a3d8d44085376be
  Stored in directory: /root/.cache/pip/wheels/d0/9c/0d/8709be17c02b72bf04ea60d8ec64fc46a1554c0fb81e048dd6
Successfully built pickle-mixin


# **Step2: Import the chatbot codes**

In [ ]:
%%writefile customer_care_chatbot.py
import streamlit as st
from streamlit_chat import message
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
from pyngrok import ngrok

# Load the pre-trained model and data
@st.cache_resource
def load_resources():
    # Load the trained model
    model = load_model('customer_care_gru_model.h5')

    # Load tokenizer
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # Load label encoder
    with open('label_encoder.pickle', 'rb') as handle:
        lbl_encoder = pickle.load(handle)

    # Load intents
    with open('intents.json') as file:
        data = json.load(file)

    return model, tokenizer, lbl_encoder, data

# Function to get bot response
def get_response(model, tokenizer, lbl_encoder, intents, text):
    # Predict the intent
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, truncating='post', maxlen=20)
    pred = model.predict(padded)
    intent = lbl_encoder.inverse_transform([np.argmax(pred)])[0]

    # Get a random response for the predicted intent
    for i in intents['intents']:
        if i['tag'] == intent:
            result = np.random.choice(i['responses'])
            break
    return result

# Main Streamlit app
def main():
    st.title("Customer Care Chatbot")
    st.write("Hello! I'm your customer care assistant. How can I help you today?")

    # Initialize session state for chat history
    if 'generated' not in st.session_state:
        st.session_state['generated'] = []
    if 'past' not in st.session_state:
        st.session_state['past'] = []

    # Load model and data
    model, tokenizer, lbl_encoder, intents = load_resources()

    # Chat input
    def get_text():
        input_text = st.text_input("You: ", "", key="input")
        return input_text

    user_input = get_text()

    if user_input:
        output = get_response(model, tokenizer, lbl_encoder, intents, user_input)
        st.session_state.past.append(user_input)
        st.session_state.generated.append(output)

    # Display chat history
    if st.session_state['generated']:
        for i in range(len(st.session_state['generated'])-1, -1, -1):
            message(st.session_state["generated"][i], key=str(i))
            message(st.session_state['past'][i], is_user=True, key=str(i) + '_user')

if __name__ == '__main__':
    main()

Writing customer_care_chatbot.py


# **Step3: Creat sample Data files**

you will need to create sample data files

In [ ]:
%%writefile intents.json
{
  "intents": [
    {
      "tag": "greeting",
      "patterns": [
        "Hi",
        "Hey",
        "Hello",
        "Good day"
      ],
      "responses": [
        "Hello! How can I help you today?",
        "Hi there! What can I do for you?",
        "Greetings! How may I assist you?"
      ]
    },
    {
      "tag": "goodbye",
      "patterns": [
        "Bye",
        "Goodbye",
        "See you later",
        "Have a nice day"
      ],
      "responses": [
        "Goodbye! Have a great day!",
        "See you later! Thanks for visiting.",
        "Bye! Come back again soon."
      ]
    },
    {
      "tag": "thanks",
      "patterns": [
        "Thanks",
        "Thank you",
        "That's helpful",
        "Appreciate it"
      ],
      "responses": [
        "You're welcome!",
        "Happy to help!",
        "Glad I could assist you!"
      ]
    },
    {
      "tag": "order_status",
      "patterns": [
        "Where is my order?",
        "Status of my order",
        "When will I get my order?",
        "Tracking my package"
      ],
      "responses": [
        "I can check your order status. Please provide your order number.",
        "For order status, please share your order ID.",
        "I'll help you track your order. What's your order number?"
      ]
    },
    {
      "tag": "refund",
      "patterns": [
        "I want a refund",
        "How do I get my money back?",
        "Return policy",
        "Refund process"
      ],
      "responses": [
        "Our refund policy allows returns within 30 days. Please provide your order number.",
        "I can help with your refund request. Could you share your order details?",
        "For refunds, we need your order information to process the request."
      ]
    },
    {
      "tag": "payment_issue",
      "patterns": [
        "Payment failed",
        "My payment didn't go through",
        "Charge declined",
        "Payment problem"
      ],
      "responses": [
        "I'm sorry you're having payment issues. Let me check this for you.",
        "Payment problems can occur due to various reasons. Let me assist you.",
        "I'll help resolve your payment issue. Please provide more details."
      ]
    }
  ]
}

Overwriting intents.json


# tokenizer.pickle and customer_care_gru_model.h5


For a complete working example, you would need to train a model first. Here's a training script you can run first:

In [ ]:
%%writefile train_model.py
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GRU
from sklearn.preprocessing import LabelEncoder
import pickle
import json

# Load intents
with open('intents.json') as file:
    data = json.load(file)

# Prepare data
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=20)

# Encode labels
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

# Model architecture
model = Sequential()
model.add(Embedding(1000, 64, input_length=20))
model.add(GRU(64, return_sequences=True))
model.add(GRU(32))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(labels), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train model
model.fit(padded_sequences, np.array(training_labels), epochs=200)

# Save model and tokenizer
model.save('customer_care_gru_model.h5')
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(lbl_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Model training complete. Files saved.")

Writing train_model.py


# **Step4: Run the chatbot**

In [ ]:
!ngrok config add-authtoken 2vR8Zvjzl8HdLPjZvriErsbtatq_2qKyvaKU9zE8XrZHXZnvT

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!nohup streamlit run customer_care_chatbot.py &
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

nohup: appending output to 'nohup.out'
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.11/json/__init__.py", line 293, in load
    return loads(fp.read(),
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [ ]:
from pyngrok import ngrok
import time
import os

# Stop previous tunnels (if any)
ngrok.kill()

# Start Streamlit app in background
os.system("streamlit run app.py &")
time.sleep(3)  # wait for server to start

# Open ngrok tunnel to Streamlit on port 8501
secure_url = ngrok.connect(8501, "http")
print(f"🔗 Your secure app is live at: {secure_url}")


🔗 Your secure app is live at: NgrokTunnel: "https://61cf-34-106-153-171.ngrok-free.app" -> "http://localhost:8501"
